In [150]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
from scipy import optimize
import inspect
import h5py

In [64]:
%matplotlib notebook

In [65]:
import sys
sys.path.append('/home/analysis_user/New_trap_code/Tools/')
import BeadDataFile
from discharge_tools import *

In [66]:
def extract_fft_peaks(data,axis,fsamp = 0, harm_freq = 0):
    freq,psd = data.psd2(axis)
    fft = np.sqrt(psd)
    
    unfiltered_peaks = signal.find_peaks(np.log(fft))[0]
    harmonics_0 = [peak for peak in unfiltered_peaks if freq[peak]%harm_freq == 0]
    harmonics_index = np.asarray([harm for harm in harmonics_0 if harmonics_0.index(harm)<50 ])
    
    return freq[harmonics_index],fft[harmonics_index]

In [67]:
def trans_func(data, axis, fit_type = 1, fsamp = 0, harm_freq = 0, w0_guess= 1, g_guess = 1, a_guess = 1, b_guess = 1):
    
    freq,psd = data.psd2(axis)
    fft = np.sqrt(psd)
    
    unfiltered_peaks = signal.find_peaks(np.log(fft))[0]
    harmonics_0 = [peak for peak in unfiltered_peaks if freq[peak]%harm_freq == 0]
    harmonics_index = np.asarray([harm for harm in harmonics_0 if harmonics_0.index(harm)<50 ])
    
    def fit_2(x,a,w,g,b):
        x_ang = 2*np.pi*x
        return a*w/np.sqrt(((w**2-x_ang**2)**2 + (g*x_ang-(b**3)/x_ang)**2))
    
    def fit_1(x,a,w,g):
        x_ang = 2*np.pi*x
        return a*w/np.sqrt(((w**2-x_ang**2)**2 + (g*x_ang)**2))
    
    if fit_type==1:
        curve = optimize.curve_fit(fit_1,freq[harmonics_index],fft[harmonics_index],p0 = [a_guess, w0_guess,g_guess])
        pars = curve[0]
        var = np.asarray([curve[1][0][0], curve[1][1][1], curve[1][2][2]])
    elif fit_type ==2:
        curve = optimize.curve_fit(fit_2,freq[harmonics_index],fft[harmonics_index],p0 = [a_guess, w0_guess,g_guess,b_guess])
        pars = curve[0]
        var = np.asarray([curve[1][0][0], curve[1][1][1], curve[1][2][2], curve[1][3][3]])
    
    return freq[harmonics_index],fft[harmonics_index],pars,var

In [68]:
def trans_func_3d(data, axis, fit_type=1, fsamp = 0, harm_freq = 0 , A = 1, B = 1, C = 1, D = 1, E = 1, F = 1, S = 1):
    freq,psd = data.psd2(axis)
    fft = np.sqrt(psd)
    
    unfiltered_peaks = signal.find_peaks(np.log(fft))[0]
    harmonics_0 = [peak for peak in unfiltered_peaks if freq[peak]%harm_freq == 0]
    harmonics_index = np.asarray([harm for harm in harmonics_0 if harmonics_0.index(harm)<50 ])
    
    def fit_1(x,s,a,b,c,d,e,f):
        w = 2*np.pi*x
        return s*np.sqrt((a-w**2)**2 + b*w**2)/np.sqrt((w**4 + c*w**2 + d)**2 + (e*w**3 + f*w)**2)
    
    def fit_2(x,s,a,b,c,d,e,f):
        w = 2*np.pi*x
        return s*np.sqrt((a + b*w**2))/np.sqrt((w**4 + c*w**2 + d)**2 + (e*w**3 + f*w)**2)
    
    
    if fit_type==1:
        curve = optimize.curve_fit(fit_1,freq[harmonics_index],fft[harmonics_index],p0 = [S,A,B,C,D,E,F])
        pars = curve[0]
    elif fit_type ==2:
        curve = optimize.curve_fit(fit_2,freq[harmonics_index],fft[harmonics_index],p0 = [S,A,B,C,D,E,F])
        pars = curve[0]
        
    return freq[harmonics_index],fft[harmonics_index],pars

In [ ]:
Kxx,Kyy,Kxy,Lxx,Lyy,Lxy,Axx,Ayy,Axy,Ayx = parameters('Kxx,Kyy,Kxy,Lxx,Lyy,Lxy,Axx,Ayy,Axy,Ayx')
fxx,fyy,fxy,fyx,Oxx,Oyy,Oxy,Oyx = variables('fxx,fyy,fxy,fyx,Oxx,Oyy,Oxy,Oyx')

Kxx.min,Kxx.max = 1e2,1e4
Kxx.value = 1e3

Kyy.min,Kyy.max = 1e2,1e4
Kyy.value = 1e3

Kxy.min,Kxy.max = 1,1e4
Kxy.value = 1


Lxx.min,Lxx.max = 1e1,1e3
Lxx.value = 1e2

Lyy.min,Lyy.max = 1e1,1e3
Lyy.value = 1e2

Lxy.min,Lxy.max = 1,1e4
Lxy.value = 1 

Axx.min,Axx.max = 0,1
Axx.value = 1

Ayy.min,Ayy.max = 0,1
Ayy.value = 1

Axy.min,Axy.max = 0,1
Axy.value = 1

Ayx.min,Ayx.max = 0,1
Ayx.value = 1

In [ ]:
def Xmodel_x(params,fxx):
    Axx,Ayy,Axy,Ayx,Kxx,Kyy,Kxy,Lxx,Lyy,Lxy = params
    numerator = 
    denominator = 

In [70]:
model = Model({
    Oxx:Axx*sp.sqrt((Kyy-(2*np.pi*fxx)**2)**2+(Lyy**2)*(2*np.pi*fxx)**2)/sp.sqrt(((2*np.pi*fxx)**4+(Lxy**2-Lxx*Lyy-Kxx-Kyy)*(2*np.pi*fxx)**2 + (Kxx*Kyy - Kxy**2))**2 + ((-Lxx-Lyy)*(2*np.pi*fxx)**3 + (Kxx*Lyy + Kyy*Lxx - 2*Kxy*Lxy)*(2*np.pi*fxx))**2),
    Oxy:Axy*sp.sqrt(Kxy**2+(Lxy**2)*(2*np.pi*fxy))/sp.sqrt(((2*np.pi*fxy)**4+(Lxy**2-Lxx*Lyy-Kxx-Kyy)*(2*np.pi*fxy)**2+(Kxx*Kyy- Kxy**2))**2 + ((-Lxx-Lyy)*(2*np.pi*fxy)**3 + (Kxx*Lyy + Kyy*Lxx - 2*Kxy*Lxy)*(2*np.pi*fxy))**2),
    Oyx:Ayx*sp.sqrt(Kxy**2+(Lxy**2)*(2*np.pi*fyx))/sp.sqrt(((2*np.pi*fyx)**4+(Lxy**2-Lxx*Lyy-Kxx-Kyy)*(2*np.pi*fyx)**2+(Kxx*Kyy - Kxy**2))**2 + ((-Lxx-Lyy)*(2*np.pi*fyx)**3 + (Kxx*Lyy + Kyy*Lxx - 2*Kxy*Lxy)*(2*np.pi*fyx))**2),
    Oyy:Ayy*sp.sqrt((Kxx-(2*np.pi*fyy)**2)**2+(Lxx**2)*(2*np.pi*fyy)**2)/sp.sqrt(((2*np.pi*fyy)**4+(Lxy**2-Lxx*Lyy-Kxx-Kyy)*(2*np.pi*fyy)**2 + (Kxx*Kyy - Kxy**2))**2 + ((-Lxx-Lyy)*(2*np.pi*fyy)**3 + (Kxx*Lyy + Kyy*Lxx - 2*Kxy*Lxy)*(2*np.pi*fyy))**2)
    
})


In [89]:
fname_x = r'/data/new_trap/20190708/Bead2/TransFunc/TransFuncX_14.h5'
bd_x= BeadDataFile.BeadDataFile(fname_x)
fname_y = r'/data/new_trap/20190708/Bead2/TransFunc/TransFuncY_16.h5'
bd_y = BeadDataFile.BeadDataFile(fname_y)
Xfreq_x,Xfft_x = extract_fft_peaks(bd_x,'x',fsamp = bd_x.fsamp, harm_freq = 23)
Xfreq_y,Xfft_y = extract_fft_peaks(bd_x,'y',fsamp= bd_x.fsamp, harm_freq = 23)
Yfreq_y,Yfft_y = extract_fft_peaks(bd_y,'y',fsamp = bd_y.fsamp,harm_freq = 23)
Yfreq_x,Yfft_x = extract_fft_peaks(bd_y,'x',fsamp = bd_y.fsamp,harm_freq = 23)